In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime


In [3]:
df = pd.read_csv('supermarket_sales - Sheet1.csv')
df.head()

,Invoice ID,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Total,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating
0,750-67-8428,A,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,1/5/2019,13:08,Ewallet,522.83,4.761905,26.1415,9.1
1,226-31-3081,C,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.8200,80.2200,3/8/2019,10:29,Cash,76.40,4.761905,3.8200,9.6
2,631-41-3108,A,Yangon,Normal,Male,Home and lifestyle,46.33,7,16.2155,340.5255,3/3/2019,13:23,Credit card,324.31,4.761905,16.2155,7.4
3,123-19-1176,A,Yangon,Member,Male,Health and beauty,58.22,8,23.2880,489.0480,1/27/2019,20:33,Ewallet,465.76,4.761905,23.2880,8.4
4,373-73-7910,A,Yangon,Normal,Male,Sports and travel,86.31,7,30.2085,634.3785,2/8/2019,10:37,Ewallet,604.17,4.761905,30.2085,5.3


In [4]:
from pandas_profiling import ProfileReport

profile = ProfileReport(
    df, title="Pandas Profiling Report for Supermarket sales dataset"
)
profile.to_file("sales_profiling.html")

/Users/hoangquocly/anaconda3/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/var/folders/56/1921flkx7_g0hh5l6jptpsxw0000gn/T/ipykernel_4357/1726412884.py:1: DeprecationWarning: `import pandas_profiling` is going to be deprecated by April 1st. Please use `import ydata_profiling` instead.
  from pandas_profiling import ProfileReport
Export report to file: 100%|██████████| 1/1 [00:00<00:00, 312.03it/s]


# Preprocessing

## Drop columns

In [5]:
df_forml = df.drop(columns=['Invoice ID', 'Tax 5%', 'cogs', 'gross margin percentage'])
df_forml.head()

,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Total,Date,Time,Payment,gross income,Rating
0,A,Yangon,Member,Female,Health and beauty,74.69,7,548.9715,1/5/2019,13:08,Ewallet,26.1415,9.1
1,C,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,80.2200,3/8/2019,10:29,Cash,3.8200,9.6
2,A,Yangon,Normal,Male,Home and lifestyle,46.33,7,340.5255,3/3/2019,13:23,Credit card,16.2155,7.4
3,A,Yangon,Member,Male,Health and beauty,58.22,8,489.0480,1/27/2019,20:33,Ewallet,23.2880,8.4
4,A,Yangon,Normal,Male,Sports and travel,86.31,7,634.3785,2/8/2019,10:37,Ewallet,30.2085,5.3


## Feature Engineering

In [7]:
# Date features
df['Date'] = pd.to_datetime(df['Date'])
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day
df['Weekday'] = df['Date'].dt.day_name()

# Time features  
df['Time'] = pd.to_datetime(df['Time']).dt.time
df['Hour'] = df['Time'].dt.hour
df['Morning'] = (df['Hour'] >= 7) & (df['Hour'] <= 11).astype(int) 
df['Afternoon'] = (df['Hour'] >= 12) & (df['Hour'] <= 16).astype(int)

# Total Revenue features
df['Revenue'] = df['Unit price'] * df['Quantity'] 

# Customer features
df['Member'] = (df['Customer type'] == 'Member').astype(int)

# Product features
product_dummies = pd.get_dummies(df['Product line'])
df = pd.concat([df, product_dummies], axis=1)

# Branch features 
branch_dummies = pd.get_dummies(df['Branch'])
df = pd.concat([df, branch_dummies], axis=1)

# City features
city_dummies = pd.get_dummies(df['City'])
df = pd.concat([df, city_dummies], axis=1)

df.head()

AttributeError: Can only use .dt accessor with datetimelike values